In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"


In [3]:
pip install pyserial opencv-python

  Obtaining dependency information for pyserial from https://files.pythonhosted.org/packages/07/bc/587a445451b253b285629263eb51c2d8e9bcea4fc97826266d186f96f558/pyserial-3.5-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/90.6 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/90.6 kB ? eta -:--:--
   ------------- -------------------------- 30.7/90.6 kB 186.2 kB/s eta 0:00:01
   --------------------------- ------------ 61.4/90.6 kB 297.7 kB/s eta 0:00:01
   ------------------------------- -------- 71.7/90.6 kB 302.7 kB/s eta 0:00:01
   ---------------------------------------- 90.6/90.6 kB 341.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#API TESTING, Sending request

In [1]:
import cv2
import serial
import time
import json
import os
import requests

def get_arduino_value(serial_port):
    """
    Reads a value from the Arduino. Returns the value as a string or None if no valid value is received.
    """
    try:
        arduino = serial.Serial(serial_port, 9600, timeout=1)
        time.sleep(1)  # Wait for the connection to stabilize
        while True:
            if arduino.in_waiting > 0:
                data = arduino.readline().decode('utf-8').strip()
                try:
                    # Convert data to float if it's a numeric value
                    value = float(data.split()[0])  # Expecting something like "SPEED:7.45"
                    arduino.close()
                    return value
                except ValueError:
                    print(f"Invalid data received: {data}")
            time.sleep(1)  # Avoid overwhelming the Arduino with constant checks
    except serial.SerialException as e:
        print(f"Error connecting to Arduino: {e}")
        return None

def record_webcam(output_file, duration=10):
    """
    Records a video from the webcam for the specified duration.
    """
    cap = cv2.VideoCapture(1)  # Open the default webcam
    if not cap.isOpened():
        print("Error: Unable to access the webcam.")
        return False

    # Get the default video frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Define the codec

    out = cv2.VideoWriter(output_file, fourcc, 20.0, (frame_width, frame_height))

    start_time = time.time()
    print("Recording...")

    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if ret:
            out.write(frame)
            # Display the video frame (optional)
            cv2.imshow('Recording', frame)

            # Press 'q' to stop early
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Error: Unable to read from webcam.")
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Video saved as {output_file}")
    return True

def send_video_to_api(video_file, speed, timestamp, api_url="https://8d24-34-125-179-30.ngrok-free.app/process"):
    """
    Sends the recorded video along with speed and timestamp to the API.
    """
    try:
        with open(video_file, 'rb') as video:
            files = {'video': video}
            json_data = {
                'speed': speed,
                'timestamp': timestamp
            }

            # Send JSON data as a form field
            response = requests.post(api_url, files=files, data={'json_data': json.dumps(json_data)})

        if response.status_code == 200:
            print("Video and metadata successfully uploaded.")
        else:
            print(f"Failed to upload video. Status code: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"Error sending video to API: {e}")
def save_metadata(video_file, speed, timestamp, output_folder="metadata"):
    """
    Save the metadata (video filename and speed) as a JSON file in the metadata folder.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    # Extract the base filename without its directory
    video_filename = os.path.basename(video_file)
    
    # Construct the metadata file path in the metadata folder
    metadata_file = os.path.join(output_folder, f"{os.path.splitext(video_filename)[0]}.json")

    metadata = {
        "video_file": video_file,
        "speed_kmph": speed,
        "timestamp": timestamp
    }

    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=4)

    print(f"Metadata saved as {metadata_file}")
    return metadata

if __name__ == "__main__":
    serial_port = "COM4"  # Replace with your Arduino's serial port
    video_index = 1  # Start video naming index
    video_folder = "video"  # Folder to store videos
    os.makedirs(video_folder, exist_ok=True)

    print("Waiting for Arduino to send a valid speed value...")

    while True:
        speed = get_arduino_value(serial_port)
        if speed is not None and speed > 4.0:  # Only record if speed is valid and above 4 km/h
            print(f"Received speed: {speed} km/h. Starting recording...")
            video_file = os.path.join(video_folder, f"output_{video_index}.avi")
            if record_webcam(video_file):
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                metadata = save_metadata(video_file, speed, timestamp)

                # Send video and metadata to the API
                send_video_to_api(video_file, speed, timestamp)

                video_index += 1  # Increment the file index for the next recording
            print("Waiting for the next speed signal from Arduino...")
        elif speed is None:
            print("No valid data received from Arduino. Exiting loop.")
            break
        else:
            print(f"Received speed: {speed} km/h, but it's below the threshold.")


Waiting for Arduino to send a valid speed value...
Received speed: 6.96 km/h. Starting recording...
Recording...
Video saved as video\output_1.avi
Metadata saved as metadata\output_1.json
Video and metadata successfully uploaded.
Waiting for the next speed signal from Arduino...
Error connecting to Arduino: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))
No valid data received from Arduino. Exiting loop.
